### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2018 ###

# Hobby Farm Production Optimization #

#### Joel Atkins (jratkins@wisc.edu), Julin Peng (jpeng49@wisc.edu), and Junyi Wei (jwei53@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Our goal was to attempt to model the decisions facing a farmer when choosing what to grow on a hobby farm.  Given a plot of land, the farmer needs to choose which crops to grow, which livestock to raise and which equipment to purchase.  Unlike large-scale commercial farms, hobby farms tend to grow a diverse mixture of crops and livestock.  There is evidence that hobby farms fail more quickly than other types of agriculture[1], for many reasons but financial difficulties appear to be the primary driver.  Optimizing the profitability of such an endeavor, while also considering the constraint of diverisity in attempt to achieve a more "idyllic" setting, increases the likelihood of success.

The complexity of this optimization problem is higher than it may seem at first glance.  On a large scale monocultural farm the problem is relatively simple, which crop has the maximum profit per acre?  On a hobby farm with a goal of diversity this is not the case.  Often, a decision about one crop or livestock is affected by another.  For example, if a farmer decides to grow a crop which can be used as feed for their animals they decrease the possible profit from that crop while also decreasing costs associated with that livestock.

Finding data with which to model this problem is not difficult.  In fact, the challenge is actually in sifting through the abundance of government and industry reports in order to track down the various values.  As our goal was attempting to create an accurate model of this problem, continuing to add more detail to bring it closer and closer to reality, government sources were identified and values for costs and returns pulled from their tables [2][3][4][5][6][7][8][9][10].  If the focus of the work had been on searching for a true optimal set of choices and maximized return these data sources would have been further validated and the values normalized to ensure cross-compatibility (e.g dates, locations, etc).

The following section will describe the iterative process used to model this problem.  Starting with a simplied "toy" model and then adding complexity in order to bring the model closer to the real world problem.  The model description is followed by [Julia] code using [JuMP] (Julia for Mathematical Optimization) in order to encode the model and find optimal results.  These results are then thoroughly described, with the final model being compared to previous, simplied iterations.  Finally, conclusions about the problem, model and process are presented.

***Citations:***

[1] Moyer, J. (2015). What nobody told me about small farming: I can’t make a living | Salon https://www.salon.com/2015/02/10/what_nobody_told_me_about_small_farming_i_cant_make_a_living/

[2] U. S. Department of Agriculture. (2017). Crop Values 2016 Summary. Washington, DC: Author.

[3] U. S. Department of Agriculture. (2018). Crop Production 2017 Summary. Washington, DC: Author.

[4] Manitoba Agriculture Farm Management. (2017). Guidelines for Estimating Crop Production Costs. Winnipeg, Manitoba: Author.

[5] Iowa State University Extension and Outreach. (2018). Estimated Returns to Farrow to Finish. Ames, IA: Author.

[6] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Yearling Steers. Ames, IA: Author.

[7] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Steer Calves. Ames, IA: Author.

[8] Iowa State University Extension and Outreach. (2018). Agriculture Decision Maker. Retrieved from https://www.extension.iastate.edu/agdm/ldcostsreturns.html

[9] UW-Madison College of Agriculture and Life Sciences. (2003). Large-scale pastured poultry farming in the U.S. Madison, WI: Author.

[10] Hamra, C.F. (2010). An Assessment of the Potential Profitability of Poultry Farms: A Broiler Farm Feasibility Case Study. Master's Thesis. University of Tennessee at Martin. Martin, TN.

   [Julia]: <https://julialang.org/>
   [JuMP]: <https://github.com/JuliaOpt/JuMP.jl>


## 2. Mathematical model ##


### 2.1 v1: Toy Model ###

We used an iterative process to produce the model for our problem.  To begin we created a "toy" LP of the form:


$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& cx \\
    \text{subject to:}\qquad& Ax \le b\\
    \end{aligned}$$


Such that $b$ is a vector of the parameters money, acres and housing available which define the specific instance of the problem.  $A$ is a matrix of monetary and space requirements for each crop/livestock choice.  And finally $c$ is a vector of profit values.

$$
{\pmatrix{a_{1,1}  & \dots & a_{1,n+m} \cr a_{2,1} & \dots & a_{2,n+m}\cr a_{3,1} & \dots  & a_{3,n+m}\cr}\atop \underset{(3 \times(n+m))}{A}}
{\pmatrix{crop_1 \\ \vdots \\ crop_n \\ livestock_1 \\ \vdots \\ livestock_m}\atop  \underset{((n+m) \times 1)}{x}}
\raise12pt\hbox{$\leq$} {\pmatrix{acres\\ housing\\ money}\atop \underset{(3 \times 1)}{b}}
$$
    
Creating this toy model allowed us to code the problem without complex constraints in order to ensure that data import and indexing was working properly.  The issue with this model is that it is an oversimplification of the problem and will only choose the single highest return crop and livestock for available acreage and housing.

### 2.2 v2: Adding Diversity Constraint, now MIP ###

Adding detail, and complexity, to our model will bring it more in line with reality.  Adding a diversity constraint, i.e. forcing the optimum solution to include a minimum number of crop choices as well as a minimum number of livestock choices, changes our Linear Program into a mixed integer program.

In order to model this constraint indicator variables were added, $y_i$ and $z_i$ which indicate whether or not a certain crop or livestock is included in the optimal choice.

$$ crop_i \  > 0 \Longrightarrow y_i = 1$$
$$ livestock_i \  > 0 \Longrightarrow z_i = 1$$

As these are conditional statements they need to be converted to algebraic constraints.  This is done with the addition of a value $M$ which is the upper bound on an optimal solution.  

$$crop_i \leq My_i $$
$$livestock_i \leq Nz_i$$

In the case of crop decisions the upper bound was set to the input parameter for available fields, in the case of livestock decisions the upper bound was set to the input parameter for available money.  These new constraints ensure that the indicator variables are valid.  Additional constraints can then be added to enforce the diversity constraint using the indicator variables.  

$$ \sum_i y_i = \text{number of desired crops}$$
$$ \sum_i z_i = \text{number of desired livestock}$$

### 2.3 v3: Adding Balance ###

In version 3 it was decided to add a balance constraint, or a lower bound for chosen items.  This was done to ensure a non-trivial amount was chosen for each crop or livestock included in the optimal choice.  In order to model this additional parameters were added.  Separate lower bounds, one for the minimum area in which to plant a chosen crop, $lb_{crops}$, and the other for the minimum number of animals for each chosen livestock species, $lb_{lstock}$.  These bounds are then enforced conditionally.

$$x_i > 0 \Longrightarrow x_i \geq \text{lower bound}$$

As indicator variables were already implemented in version 2 they can be used in order to implement additional constraints which enforce these lower bounds.

$$ crop_i \geq y_i \cdot lb_{crops}$$
$$ livestock_i \geq z_i \cdot lb_{lstock}$$



### 2.4 v4: ... ###


## 3. Solution ##

The following code represents our iterative process of creating a realistic model.  This process was borrowed from a core concept in software engineering.  Start by developing core functionality and then add features one by one.  Each version implemented built upon the previous version.  The output of each model was analyzed to determine where it differed from the conceptualized ideal model.  This was followed by adding complexity in order to better model the desired problem.

### 3.1 Preparation: Data Import ###

Data was collected from various sources (as outlined in section 1) and input into 2 CSV files, one for crops and one for livestock.  These CSV files contain info on various costs, yields and returns for the possible crop and livestock choices a farmer can make.  These various values are consolidated into columns which provide costs and gross returns for each of the possible choices.  This information is imported into well formatted arrays with the following code:

In [ ]:
using DataFrames, CSV, NamedArrays
raw = CSV.read("crops1.csv");
# turn Crops into an array
crops_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each crop;
info_crops = names(raw[2:end]);
# create a list of crops from the crops array
crops = crops_array[1:end,1];
# create a NamedArray that specifies the information of each crop
crops = convert(Array{Symbol},crops )
using NamedArrays
crops_matrix = crops_array[1:end,2:end]; # rows are crops, columns are information of crops
crops_information_array = NamedArray(crops_matrix, (crops, info_crops), ("crops","info_crops"));


raw1 = CSV.read("LiveStock.csv");
# turn LiveStock into an array
lstocks_array = convert(Array,raw1);
# the `names' of the DataFrame (header) are the information of each livestock;
info_lstocks = names(raw1[2:end]);
# create a list of stocks from the livestocks array
lstocks = lstocks_array[1:end,1];
# create a NamedArray that specifies the information of each livestock
lstocks = convert(Array{Symbol},lstocks )
lstocks_matrix = lstocks_array[1:end,2:end] ;# rows are livestocks, columns are information of livestocks
lstocks_information_array = NamedArray(lstocks_matrix, (lstocks, info_lstocks), ("lstocks","info_lstocks"));

### 3.2 v1: Toy Model ###
As described in the Math Model section our first iteration was a, relatively, simple linear program which attempted to maximize net return on our farm.  This was modeled with the following code and solved with CLP:

In [ ]:
#the properties of our farm
tt_field=900;
tt_housing=3;
tt_money=100000
using JuMP, Clp
m = Model(solver=ClpSolver()) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, lstock[lstocks]>=0)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)
#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression for the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestock
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return ] for i in lstocks))
#expression for the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))

### 3.3 v2: Diversity Constraint###

The second version of the model added a diversity constraint.  This required adding binary indicator variables $y_i$ and $z_i$ which were set, using an upper bound, to indicate whether or not $crop_i$ or $livestock_i$ was chosen.  This was followed by adding a constraint such that the sum of $y_i$s was equal to $numcrop$, or desired number of crops, and the sum of $z_i$s was equal to numlstock, or desired number of types of animals.  The transition to a non-linear solver also allowed greater realism by restricting livestock choices to integer values.

In [ ]:
#the properties of our farm
tt_field=900;
tt_housing=3;
tt_money=10000000
#diversity constraint for number of crops to choose
numcrop=5       ### Added in V2
#diversity constraint for number of types of animals to choose
numlstock=5     ### Added in V2

using JuMP,Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0)) # create model named m, now using Gurobi for MIP

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, y[crops], Bin)
@variable(m, lstock[lstocks], Int)
@variable(m, z[lstocks], Bin)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)

#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

# if crop[i]>0, then y[i]=1
@constraint(m,div1[i in crops], crop[i]<=y[i]*tt_field)     ### Added in V2

# if lstock[i]>0, then z[i]=1
@constraint(m, div2[j in lstocks], lstock[j]*lstocks_information_array[j,:Cost]<=z[j]*tt_money)     ### Added in V2

#diversity constraint for crop
@constraint(m, sum(y[i] for i in crops)==numcrop)     ### Added in V2

#diversity constraint for livestock
@constraint(m,sum(z[j]for j in lstocks)>=numlstock)     ### Added in V2


#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression for the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestock
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return] for i in lstocks))
#expression for the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))
println(getvalue(y))
println(getvalue(z))


### 3.4 v3: Adding Balance ###
The third version of the model added a balance constraint, a lower bound on the amount of each chosen crop/livestock.  This required adding parameters defining the lower bounds.  This was followed by adding a constraint such that for each crop or livestock the amount chosen was equal to the indicator variable multiplied with the lower bound.  As the indicator variable for items not chosen is 0 the bound is not enforced in those cases.

In [ ]:
#the properties of our farm
tt_field=900;
tt_housing=90000;
tt_money=500000
numcrop=5
numlstock=5
# lower bound on area in which a chosen crop must be planted
lb_crops =tt_field /(2*numcrop)     ### Added in v3
# lower bound on number of head of each chosen type of animal
lb_lstocks =2     ### Added in v3


using JuMP,Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0)) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, y[crops], Bin)
@variable(m, lstock[lstocks], Int)
@variable(m, z[lstocks], Bin)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)

#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

# if crop[i]>0, then y[i]=1
@constraint(m,div1[i in crops], crop[i]<=y[i]*tt_field)

# if lstock[i]>0, then z[i]=1
@constraint(m, div2[j in lstocks], lstock[j]*lstocks_information_array[j,:Cost]<=z[j]*tt_money)

#if y[i]=1, then crop[i]>lbcrops
@constraint(m,lb1[i in crops], crop[i]>=y[i]*lb_crops)     ### Added in v3

#if z[i]=1, then lstock[i]>lblstocks
@constraint(m,lb2[i in lstocks], lstock[i]>=z[i]*lb_lstocks)     ### Added in v3

#diversity constraint for crop
@constraint(m, sum(y[i] for i in crops)>=numcrop)

#diversity constraint for livestock
@constraint(m,sum(z[j]for j in lstocks)>=numlstock)

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression fpr the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestocks
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return ] for i in lstocks))
#expression fpr the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))
println(getvalue(y))
println(getvalue(z))

### 3.5 v4: ... ###

## 4. Results and discussion ##

The results of each iteration were analyzed to provide insight into which detail would need to be added to the model in order to add more realism and derive more utility.  The exact results are data and parameter dependent and individual choices in the results do not indicate maximum profit outside the confines of this model.

### 4.1 v1 Results ###

As could be predicted the "toy" LP output an optimal solution in which just the most profitable selections were chosen to utilize all available acreage and housing.  In this case all acreage was planted with sunflowers and all housing utilized for broiler chickens.  This simple model, though, allowed us to manipulate input parameters in order to find changes in output and gain a better understanding of model sensitivity.

| v1 Optimal       | Acreage       | Housing  |
| ------------- |:-------------| -----:|
| Sunflowers     |100% |  |
| Broiler Chickens| | 100%|

### 4.2 v2 Results ###

Given the constraints and objective of the "toy" linear program there can only be one choice to utilize each input parameter unless multiple options have the same cost, return and requirements.  The stated objective of this projected is to attempt to model hobby farming decisions with the desired goal of a profitable and diverse small farm.  In order to force diverse choices in our model it was necessary to implement a minimum number of choices in both crops and livestock.  This, now MIP, model again produced expected results.  The optimal choices determined by the LP were maximized while satisfying the constraint by making additional choices but at such small numbers as to be effectively 0.

| v2 Optimal       | Acreage       | Housing  |
| ------------- |:-------------| -----:|
| Sunflowers     |~100% |  |
|2 | ~0%| |
|3 | ~0%| |
|4 | ~0%| |
|5 | ~0%| |
| Broiler Chickens| | ~100%|
|7| | 1|
|8| | 1|
|9| | 1|
|10| | 1|

### 4.3 v3 Results ###

Adding a requirement for a minimum number of choices was not enough to make the model select for a truly diverse set of crops and livestock.  This diversity constraint is not something easily codified into mathematical constraints and additional restrictions are necessary in order to bring the model inline with the subjective goal.  Lower bounds can force the model away from choosing to commit nearly 100% of the resources to the most profitable choices.  The strength of optimality of the choices by the original LP are still evident in the fact that suboptimal choices will be allocated resources exactly equal to the lower bound with all remaining allocated to the optimal choice.

| v3 Optimal       | Acreage       | Housing  |
| ------------- |:-------------| -----:|
| Sunflowers     |total - (4 * crop lower bound) |  |
|2 | crop lower bound| |
|3 | crop lower bound| |
|4 | crop lower bound| |
|5 | crop lower bound| |
| Broiler Chickens| | total - (4 * livestock lower bound)|
|7| | livestock lower bound|
|8| | livestock lower bound|
|9| | livestock lower bound|
|10| | livestock lower bound|

### 4.4 v4 Results ###

## 5. Conclusion ##

This project was a valuable exercise in understanding various LP and MIP models.  Adjustment of parameters permitted greater insight into the sensitivity of the model.  In a real-world scenario the input parameters representing available resources would be fixed, so the sensitivity to parameter adjustmets would be restricted to diversity and balance choices. In addtion, in order to derive realistic insight from the results of the models a great effort in data validation and normalization would be required, well beyond the scope of this assignment.

Another interesting insight gained through the exercise is the connection between building out the code which defines the model and the resulting impact on the model itself.  While the code can be thought of as growing the model itself can be thought of as shrinking to better fit the true reality.  Each additional constraint grows the codebase, but most will shrink the feasible space.

A future direction to pursue in enhancing the model could be to include an attempt at predictive returns based upon past market data.  Again, this would require great effort in data collection and validation, but the end goal would be a model which utilizes historical year-to-year data in order to predict which crops and livestock will provide the greatest returns in the upcoming year.